In [1]:
from keras.models import Model
from keras.layers import Dense, LSTM, Input
from keras.optimizers import SGD
from keras.losses import mean_squared_error
from keras.activations import relu
import numpy as np
import random
from sklearn.metrics import mean_absolute_error

Using TensorFlow backend.


In [2]:
train_X = np.random.uniform(size=(100000, 10, 1))
train_y = np.std(train_X, axis=1) * 10
test_X = np.random.uniform(size=(1000, 10, 1))
test_y = np.std(test_X, axis=1) * 10

In [3]:
input_ = Input(shape=(10, 1))
lstm1 = LSTM(32, return_sequences=True)(input_)
lstm2 = LSTM(32)(lstm1)
output = Dense(1)(lstm2)

model = Model(inputs=input_, outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 1)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 32)            4352      
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 12,705
Trainable params: 12,705
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.compile(SGD(lr=0.1), loss=mean_squared_error)

In [5]:
model.fit(train_X, train_y, epochs=10)

Epoch 1/10
100000/100000 [==============================] - 26s 263us/step - loss: 0.2097
Epoch 2/10
100000/100000 [==============================] - 26s 256us/step - loss: 0.2052
Epoch 3/10
100000/100000 [==============================] - 26s 262us/step - loss: 0.2045
Epoch 4/10
100000/100000 [==============================] - 26s 262us/step - loss: 0.2035
Epoch 5/10
100000/100000 [==============================] - 27s 265us/step - loss: 0.0871
Epoch 6/10
100000/100000 [==============================] - 27s 265us/step - loss: 0.0101
Epoch 7/10
100000/100000 [==============================] - 26s 263us/step - loss: 0.0074
Epoch 8/10
100000/100000 [==============================] - 27s 272us/step - loss: 0.0062
Epoch 9/10
100000/100000 [==============================] - 27s 270us/step - loss: 0.0053
Epoch 10/10
100000/100000 [==============================] - 27s 270us/step - loss: 0.0048


In [6]:
test_pred = model.predict(test_X)